In [ ]:
import json
import os
import sys
import timeit

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import mean_squared_error

# sys.path.append(r"../")
from ct_assist import transform
from ct_assist.utils import accuracy

# Testing for optimal iterations vs. execution time

In [ ]:
results = []
results_time = []
begin = 2
end = 5
time_settings = np.logspace(begin,end,num=end-begin+1,base=10,dtype='int')

In [ ]:
time_settings

In [ ]:
image_coord_dict = dict()
image_coord_dict["00.jpg"] = np.array([[1470, 1430], [2333, 1355], [3247, 1781], [1935, 1952]])
image_coord_dict["01.jpg"] = np.array([[1495, 1552], [2219, 1589], [1843, 1969], [805, 1875]])
image_coord_dict["03.jpg"] = np.array([[1216, 1398], [2215, 1754], [3268, 1530], [2067, 1282]])   

def setup_vars():
    """Loads data for test_transform_image"""
    data_dir = r"./data/table"
    json_fp = os.path.join(data_dir, "anno.json")
    arr_fp = os.path.join(data_dir, "anno.npz")
    with open(json_fp, "r") as fp:
        mappings = json.load(fp)

    with np.load(arr_fp) as arrs:
        anno_dict = {img: {"heads": arrs[f"{prefix}heads"],
                           "feet": arrs[f"{prefix}feet"]}
                     for img, prefix in mappings.items()}
    
    for key, items in anno_dict.items():
        if key.endswith("02.jpg"):
            continue
        else:
            image_coords = image_coord_dict[key[-6:]]
        # feet and heads have been swapped in annotations
        reference = np.array([items["feet"], items["heads"]])
        height = 0.095  # m
        STD = 0.01  # m
        img = Image.open(key)
        yield (img, reference, height, STD, image_coords)
        
params = next(setup_vars())

Approximately linear increase 

In [ ]:
number = 10

for setting in time_settings:
    func = lambda: results.append(accuracy.calc_area(transform.fit_transform(*params, iters=setting)[:,:2]))
    results_time.append(timeit.timeit(func, number=int(number)))
    print(f"{setting} iterations takes approx {results_time[-1] / number} seconds")

In [ ]:
# convert to np.array
results_time = np.array(results_time)
results = np.array(results)

In [ ]:
results_time / number

In [ ]:
results

In [ ]:
y_true = np.repeat(1.33455, results.size)

In [ ]:
RMSE = []
for i in range(0, number * time_settings.size, number):
    RMSE.append(mean_squared_error(y_true[:number], results[i:i+number], squared=False))
    print(RMSE[-1])
    
    
# mean_squared_error(y_true, results, squared=False)

In [ ]:
plt.scatter(results_time / number, RMSE)
plt.plot(results_time / number, RMSE)
plt.show()

For small scale experiments, 1e4 seems to be the optimal time-performance option. 